# 참고
- https://pytorch.org/docs/0.4.0/nn.html
- https://pytorch.org/tutorials/
- https://ratsgo.github.io/machine%20learning/2017/10/12/terms/
- https://github.com/DSKSD/Pytorch_Fast_Campus_2018

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
from collections import OrderedDict

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# What is a PyTorch?
- Tensor와 Optimizer, Neural Net등 GPU연산에 최적화된 모듈을 이용하여 빠르게 딥러닝 모델을 구현할 수 있는 프레임워크
Facebook이 밀고 있던 lua기반의 torch를 python버전으로 포팅함.

# > Pytorch Basic
- Tensor
- autograd
- nn
- optim

## 2. autograd
- 딥러닝 프레임워크의 가장 큰 장점은 자동으로 미분을 계산해주는 것임.
- 딥러닝에서 forward 단계에서 autograd은 수행하는 모든 연산을 기억함. 그리고 역전파 단계에서  미분을 계산한.

### 2.1 Tensor
- autograd에서 requires_grad=True로 설정되면 Tensor의 연산은 기록됨. 역전파(backward)연산 후에는 이 변수에 대한 gradient(변화도)는 .grad에 누적됨.
- autorgrad구현에서 Function클래스도 매우 중요한데 Tensor와 Function은 상호 연결되어 있으며 모든 연산 과정을 부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성함. 각 변수는 .grad_fn속성을 갖는데 이는 생성한 Function을 참조하고 있음.(단, 사용자가 만든 Tensor는 예외로, 이 때 grad_fn은 None)임.
- gradient를 계산하기 위해서는, Tensor의 .backward()를 호출하면 됨. 

In [3]:
# tensor를 생성하고 연산을 추적하기 위해 requries_grad=True로 설정함.
x= torch.ones(2,2, requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [4]:
print(x.data)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [5]:
print(x.grad) # 현재 gradient를 계산하지 않음

None


In [6]:
print(x.grad_fn) # 생성된 텐서임

None


In [7]:
# x에 대해 연산을 수행.
y= x+2
print(y)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


In [8]:
# 연산의 결과로 생성된 것이므로 grad_fn을 가짐.
print(y.grad_fn)

In [21]:
# y에 다른 연산을 수행
z= y*y*3
out= z.mean()
print(z)
print(out)

tensor([[  3.6049,   8.3441],
        [  3.2794,  22.9086]])
tensor(9.5343)


In [22]:
z.grad_fn

### 2.2 변화도
- backwar에서 Tensor가 Scalar인 경우는 backward에 인자를 정해줄 필요가 없음.  

In [9]:
print(out)
out.backward()
print(x.grad)

tensor(27.)
tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


$$out= \frac{1}{4}\sum_{i}z_{i}$$  
$$z_{i}= 3(x_{i}+2)^{2}$$  
$$\frac{\partial{out}}{\partial{z_{i}}}=\frac{z_{i}}{4}$$  
$$\frac{\partial{z_{i}}}{\partial{x_{i}}}=6(x_{i}+2)$$  
$$\frac{\partial{out}}{\partial{x_{i}}}=\frac{\partial{out}}{\partial{z_{i}}} * \frac{\partial{z_{i}}}{\partial{x_{i}}} = \frac{z_{i}}{4}*6(x_{i}+2) =\frac{3}{2}(x_{i}+2)$$  
$$\frac{\partial{out}}{\partial{x_{i}}}\mid_{x_{i}=1}= \frac{9}{2}$$  

- 하지만 여러개의 요소를 가질 때는 Tensor의 모양을 gradient의 인자로 지정해줄 필요가 있음.
- 인자로 들어간 Tensor가 값이 1일 때는 순수한 gradient이고 값이 k일 때는 gradient*k가 계산됨

In [10]:
x= torch.ones(2,2, requires_grad=True)
y= 2*x+1
try:
    y.backward()
except Exception as e:
    print(e)

grad can be implicitly created only for scalar outputs


In [11]:
print(y)x

tensor([[ 3.,  3.],
        [ 3.,  3.]])


In [14]:
torch.ones(2,2)

tensor([[ 1.,  1.],
        [ 1.,  1.]])

In [12]:
x= torch.ones(2,2, requires_grad=True)
y= 2*x+1
try:
    y.backward(torch.ones(2,2))
except Exception as e:
    print(e)
print(x.grad)

tensor([[ 2.,  2.],
        [ 2.,  2.]])


In [15]:
torch.ones(2,2)*2

tensor([[ 2.,  2.],
        [ 2.,  2.]])

In [16]:
x= torch.ones(2,2, requires_grad=True)
y= 2*x+1
try:
    y.backward(torch.ones(2,2)*3)
except Exception as e:
    print(e)
    
print(x.grad)

tensor([[ 6.,  6.],
        [ 6.,  6.]])


- 기본적으로 변화도 연산은 그래프 상의 모든 내부 버퍼를 새로쓰기(flush) 떄문에, 그래프의 특정 부분에 대해서 역전판 연산을 2번하고 싶다면, 첫 연산 단계에서 retain_variables=True의 값을 넘겨줘야함.

In [17]:
x= torch.randn(2,2, requires_grad=True)
y= 2*x+2

z= y*y+2
z= z.sum()
try:
    z.backward() # retain_graph=True로 하면 내부 버퍼가 사라지는 것을 막아줌.

    print(x.grad)

    z.backward()
    print(x.grad)
except Exception as e:
    print(e)

tensor([[ 13.2908,  10.1354],
        [  8.4934,  12.9705]])
Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.


In [18]:
x= torch.randn(2,2, requires_grad=True)
y= 2*x+2

z= y*y+2
z= z.sum()
try:
    z.backward(retain_graph=True) # retain_graph=True로 하면 내부 버퍼가 사라지는 것을 막아줌.

    print(x.grad)

    z.backward()
    print(x.grad)
except Exception as e:
    print(e)

tensor([[  4.3848,   6.6710],
        [ -4.1821,  11.0535]])
tensor([[  8.7695,  13.3419],
        [ -8.3643,  22.1069]])


- with torch.no_grad():로 코드 블럭을 사용하면 autograd가 requires_graph=True인 Tensor들의 연산 기록을 추적을 제외할 수 있음.

In [19]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False
